In [15]:
##### FROM https://github.com/mdhabibi/LIME-for-Time-Series

import sys
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import copy
from sklearn.metrics import pairwise_distances
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.utils import class_weight
from sklearn import metrics
from sklearn.metrics import RocCurveDisplay
from sklearn.utils import class_weight
from sklearn.model_selection import StratifiedKFold
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.utils import class_weight
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from numpy import argmax

from tensorflow.keras.utils import to_categorical

In [ ]:
X=pd.read_csv('X.csv')
y=pd.read_csv('y.csv')

In [ ]:
y_one_hot = to_categorical(y, num_classes=2)
X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot, test_size=0.1, stratify=y_one_hot, shuffle=True, random_state=10)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, stratify=y_train, shuffle=True, random_state=10)
scaler = preprocessing.StandardScaler()
X_train = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
X_test = scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)
X_val = scaler.transform(X_val.reshape(-1, X_val.shape[-1])).reshape(X_val.shape)

def make_model(input_shape,units):
    input_layer = keras.layers.Input(input_shape)

    conv1 = keras.layers.Conv1D(filters=int(units[0]), kernel_size=3, padding="same")(input_layer)
    conv1 = keras.layers.BatchNormalization()(conv1)
    conv1 = keras.layers.ReLU()(conv1)

    conv2 = keras.layers.Conv1D(filters=int(units[1]), kernel_size=3, padding="same")(conv1)
    conv2 = keras.layers.BatchNormalization()(conv2)
    conv2 = keras.layers.ReLU()(conv2)

    conv3 = keras.layers.Conv1D(filters=int(units[2]), kernel_size=3, padding="same")(conv2)
    conv3 = keras.layers.BatchNormalization()(conv3)
    conv3 = keras.layers.ReLU()(conv3)

    x = keras.layers.GlobalAveragePooling1D()(conv3)

    output_layer = keras.layers.Dense(2, activation="softmax")(x)
    return keras.models.Model(inputs=input_layer, outputs=output_layer)

model = make_model((X.shape[1],X.shape[2]),[64,128,64])
class_weights = class_weight.compute_class_weight(
                                                      class_weight = "balanced",
                                                      classes = np.unique(y_train[:,0]),
                                                      y = y_train[:,0]
                                                  )
class_weights = dict(zip(np.unique(y_train[:,0]), class_weights))

epochs = 150
batch_size = 128
optimizer = keras.optimizers.Adam(learning_rate=float(0.01))
model.compile(  optimizer='adam',
                loss= "binary_crossentropy",
                    metrics=[
                    keras.metrics.AUC(),
                ],)
callbacks = [keras.callbacks.ReduceLROnPlateau(
                monitor="val_loss", factor=0.5, patience=25, min_lr=0.0001    ),
            keras.callbacks.EarlyStopping(monitor="val_loss", patience=25, verbose=1),]        # Train the CNN model
model.fit( X_train,
                  y_train,
                  batch_size=batch_size,
                  epochs=epochs,
                  callbacks=callbacks,
                  validation_data=(X_val,y_val),
                  verbose=1,
                  class_weight=class_weights,
              )

X_test.shape

# save the trained model
model.save('model')


# Create the probability vector for each instances in test dataset
probability_vectors = model.predict(X_test)
print("Shape of probability_vector (number of rows in test dataset, number of classes):", probability_vectors.shape)
print("Predicted Classes:", probability_vectors)

print('ROC AUC:', roc_auc_score(y_test,probability_vectors))

In [1]:
##### FROM https://github.com/mdhabibi/LIME-for-Time-Series

def generate_random_perturbations_matrix(num_perturbations, num_slices,channels):
    """
    Generates random perturbations for ECG signal segments.
    
    This function creates a binary matrix where each row represents a perturbation,
    and each column corresponds to a segment of the ECG signal. A value of '1' indicates
    the segment is active or unchanged, while '0' indicates the segment is inactive or altered.
    
    Parameters:
        num_perturbations (int): The number of perturbations to generate.
        num_slices (int): The number of slices (segments) each ECG signal is divided into.
        
    Returns:
        np.ndarray: A binary matrix representing random perturbations.
    """
    random_perturbations = np.random.binomial(1, 0.5, size=(num_perturbations, num_slices, channels))
    return random_perturbations



import copy



def analyze_prediction(probability_vector, class_labels):
    """
    Analyzes the probability vector from a model prediction, returning the top predicted classes
    and the most likely predicted class.
    
    Parameters:
        probability_vector (np.ndarray): The probability vector for a given instance, as predicted by the model.
        class_labels (list): A list of class labels, adjusted to be zero-based.
    
    Returns:
        tuple: A tuple containing a list of the top predicted classes and the most likely predicted class.
    """
    # Sort the classes based on the probability vector and select the top N classes
    top_pred_classes = probability_vector[0].argsort()[-len(class_labels):][::-1]
    
    # Use np.argmax to find the index of the maximum value in the probability vector
    predicted_class_index = np.argmax(probability_vector, axis=1)
    
    # Map the predicted index to its corresponding class label
    predicted_classes = [class_labels[i] for i in predicted_class_index]
    
    # Since we're predicting for one instance, access the first element for the predicted class
    predicted_class = predicted_classes[0]
    
    return top_pred_classes, predicted_class


def segment_ecg_signal(instance_ecg, num_slices=40):
    """
    Segments an ECG signal into a fixed number of slices.

    Parameters:
        instance_ecg (np.ndarray): The ECG signal instance to segment.
        num_slices (int): The number of slices to divide the signal into.

    Returns:
        int: The width of each slice in the segmented ECG signal.
    """
    total_length = len(instance_ecg)
    slice_width = total_length // num_slices
    return slice_width

def perturb_total_mean(signal, start_idx, end_idx):
    """
    Perturbs a segment of the signal by replacing it with the overall mean of the signal.
    
    Parameters:
        signal (np.ndarray): The original signal to perturb.
        start_idx (int): The starting index of the segment to perturb.
        end_idx (int): The ending index of the segment to perturb.
        
    Returns:
        np.ndarray: The signal with the specified segment perturbed by the total mean.
    """
    modified_signal = signal.copy()
    modified_signal[start_idx:end_idx] = modified_signal.mean()
    return modified_signal
def perturb_mean_channel(signal, start_idx, end_idx, channel):
    """
    Directly modifies a segment of the signal by replacing it with the mean of that segment. Accepts multichannel signals
    
    Parameters:
        signal (np.ndarray): The signal to perturb, modified in place.
        start_idx (int): The starting index of the segment to perturb.
        end_idx (int): The ending index of the segment to perturb.
    """
    mean_value = np.mean(signal[start_idx:end_idx,channel])
    signal[start_idx:end_idx,channel] = mean_value
def perturb_mean(signal, start_idx, end_idx):
    """
    Directly modifies a segment of the signal by replacing it with the mean of that segment.
    
    Parameters:
        signal (np.ndarray): The signal to perturb, modified in place.
        start_idx (int): The starting index of the segment to perturb.
        end_idx (int): The ending index of the segment to perturb.
    """
    mean_value = np.mean(signal[start_idx:end_idx])
    signal[start_idx:end_idx] = mean_value

def perturb_noise(signal, start_idx, end_idx):
    """
    Perturbs a segment of the signal by replacing it with random noise within the signal's range.
    
    Parameters:
        signal (np.ndarray): The original signal to perturb.
        start_idx (int): The starting index of the segment to perturb.
        end_idx (int): The ending index of the segment to perturb.
        
    Returns:
        np.ndarray: The signal with the specified segment perturbed by random noise.
    """
    modified_signal = signal.copy()
    modified_signal[start_idx:end_idx] = np.random.uniform(modified_signal.min(), modified_signal.max(), end_idx - start_idx)
    return modified_signal



def apply_perturbation_to_ecg_channel_matrix(signal, perturbation, num_segments, channels,perturb_function=perturb_mean_channel):
    """
    Apply a perturbation to an ECG signal with multiple channels.

    Parameters:
    - signal (np.ndarray): The original ECG signal.
    - perturbation (np.ndarray): A vector indicating which segments to turn on (1) or off (0).
    - num_segments (int): The total number of segments the ECG signal is divided into.
    - perturb_function (function): The function to use for perturbing the signal (default is perturb_mean).

    Returns:
    - np.ndarray: A perturbed version of the ECG signal.
    """
    # Copy the signal to avoid modifying the original
    perturbed_signal = copy.deepcopy(signal)
    segment_length = len(signal) // num_segments
    
    
    
    
    
    
    for j in range(channels):
    # Apply the perturbation based on the provided vector
        for i, active in enumerate(perturbation[:,j]):
            start_idx = i * segment_length
            end_idx = start_idx + segment_length
            # Apply perturbation function only to "off" segments
            if not active:
                perturb_mean_channel(perturbed_signal, start_idx, end_idx, j)

    return perturbed_signal



def predict_perturbations(model, instance_ecg, random_perturbations, num_slices, perturb_function, channel, channels):
    """
    Applies a set of perturbations to an ECG signal, predicts the class probabilities for each perturbed signal,
    and collects the predictions.

    Parameters:
    - model: The trained ECG classifier model.
    - instance_ecg (np.ndarray): The original ECG signal instance.
    - random_perturbations (np.ndarray): An array of perturbation vectors.
    - num_slices (int): The total number of segments the ECG signal is divided into.
    - perturb_function (function): The function to use for perturbing the signal (e.g., perturb_mean).

    Returns:
    - np.ndarray: An array of model predictions for each perturbed ECG signal.
    """
    perturbation_predictions = []

    for perturbation in random_perturbations:
        # Apply the current perturbation to the ECG signal
        perturbed_signal = apply_perturbation_to_ecg_channel_matrix(instance_ecg, perturbation, num_slices,channels, perturb_function)

        # Reshape as required by the model
        perturbed_signal_reshaped = perturbed_signal.reshape(1, len(perturbed_signal), channels)  

        # Predict the class probabilities
        model_prediction = model.predict(perturbed_signal_reshaped)
        perturbation_predictions.append(model_prediction)

    # Convert the list of predictions into a numpy array
    perturbation_predictions = np.array(perturbation_predictions)
    return perturbation_predictions

def calculate_cosine_distances(random_perturbations, num_slices):
    """
    Calculates the cosine distances between each perturbation vector and the original signal representation.

    Parameters:
    - random_perturbations (np.ndarray): An array of perturbation vectors.
    - num_slices (int): The total number of segments the ECG signal is divided into, matching the dimension of the perturbations.

    Returns:
    - np.ndarray: An array of cosine distances for each perturbation from the original signal representation.
    """
    random_perturbations_flattened=np.zeros((random_perturbations.shape[0],random_perturbations.shape[1]*8))
    for j in range(random_perturbations.shape[0]):
        random_perturbations_flattened[j,:]= random_perturbations[j,:,:].ravel()   
        
    # Represent the original ECG signal as a perturbation where all segments are enabled (i.e., a vector of ones)
    original_ecg_rep = np.ones((1, num_slices*8))

    # Calculate cosine distances
    cosine_distances = pairwise_distances(random_perturbations_flattened, original_ecg_rep, metric='cosine').ravel()

    return cosine_distances

def calculate_weights_from_distances(cosine_distances, kernel_width=0.25):
    """
    Applies a kernel function to cosine distances to calculate weights for each perturbation.

    Parameters:
    - cosine_distances (np.ndarray): An array of cosine distances for each perturbation from the original signal representation.
    - kernel_width (float): The kernel width parameter for the exponential kernel function. Default is 0.25.

    Returns:
    - np.ndarray: An array of weights for each perturbation, derived from the cosine distances.
    """
 
    weights = np.sqrt(np.exp(-(cosine_distances ** 2) / kernel_width ** 2))
    return weights

def fit_explainable_model(perturbation_predictions, random_perturbations, weights, target_class):
    """
    Fits a linear regression model to quantify the importance of each segment 
    in the decision-making process for the target class.

    Parameters:
    - perturbation_predictions (np.ndarray): The array of model predictions for each perturbed ECG signal.
    - random_perturbations (np.ndarray): The matrix of perturbation vectors.
    - weights (np.ndarray): The array of weights corresponding to each perturbation.
    - target_class (int): The index of the target class to explain.

    Returns:
    - np.ndarray: The coefficients of the linear regression model, indicating the importance of each segment.
    """
    # Initialize the linear regression model
    explainable_model = LinearRegression()

    # Squeeze the middle dimension out from perturbation_predictions to get a 2D array
    perturbation_predictions_squeezed = np.squeeze(perturbation_predictions, axis=1)

    # Select the predictions for the target class across all perturbations
    target_predictions = perturbation_predictions_squeezed[:, target_class]
    
    random_perturbations_flattened=np.zeros((random_perturbations.shape[0],random_perturbations.shape[1]*8))
    for j in range(random_perturbations.shape[0]):
        random_perturbations_flattened[j,:]= random_perturbations[j,:,:].ravel()
    # Fit the model
    explainable_model.fit(X=random_perturbations_flattened, y=target_predictions, sample_weight=weights)

    # Extract the coefficients
    segment_importance_coefficients = explainable_model.coef_

    return segment_importance_coefficients


def identify_top_influential_segments(segment_importance_coefficients, number_of_top_features=5):
    """
    Identifies the top influential segments of an ECG signal based on the importance coefficients 
    obtained from a linear regression model.

    Parameters:
    - segment_importance_coefficients (np.ndarray): The coefficients indicating the importance of each segment.
    - number_of_top_features (int): The number of top influential segments to identify.

    Returns:
    - np.ndarray: Indices of the top influential segments.
    """
    # Sort the coefficients based on their absolute magnitude to identify the most influential segments
    top_influential_segments = np.argsort(np.abs(segment_importance_coefficients))[-number_of_top_features:]
    
    return top_influential_segments

In [14]:
def get_segments(  model, X_test, id_ecg ,electrodo,number_of_top_features,num_perturbations = 200,num_slices = 50, channels=8):
    
    #model= Trained model
    #X_test= test dataset
    #id_ecg= number of trial in test dataset
    #electrodo= list of electrodes names
    #number_of_top_features= desired number of top features
    #num_perturbations= number of random perturbations applied to the signals
    #num_slices= number of time segments the signals get split into
    
    instance_ecg = X_test[id_ecg, :]     

    # Predict the class of the selected signal by trained model
    probability_vector = model.predict(instance_ecg[np.newaxis, :])

    # class labels
    class_labels = [0, 1]

    print(instance_ecg.shape)
    top_pred_classes, predicted_class = analyze_prediction(probability_vector, class_labels)


    slice_width = segment_ecg_signal(instance_ecg, num_slices)

    random_perturbations = generate_random_perturbations_matrix(num_perturbations, num_slices,channels)
    
    
    perturbation_predictions = predict_perturbations(model, instance_ecg, random_perturbations, num_slices, perturb_mean_channel,0,channels)

# Calculate cosine distances between each perturbation and the original ECG signal representation
    cosine_distances = calculate_cosine_distances(random_perturbations, num_slices)

#Applying a Kernel Function to Compute Weights
    kernel_width = 0.25  # This can be adjusted based on your specific needs
    weights = calculate_weights_from_distances(cosine_distances, kernel_width)

    # Now we have the weights for each perturbation for further analysis

# Constructing the Explainable Model for ECG Signals
    segment_importance_coefficients = fit_explainable_model(perturbation_predictions, random_perturbations, weights, target_class=top_pred_classes[0])

    # The importance coefficients for each segment
# Isolating Influential Signal Segments
    print(segment_importance_coefficients.shape)
    top_influential_segments = identify_top_influential_segments(segment_importance_coefficients, number_of_top_features)

# The indices of the top influential segments
    print("Top Influential Signal Segments:", top_influential_segments)
# Visualizing the LIME Explanation: Highlighting Key Segments
    results= np.zeros((channels,num_slices))
    for i in range(channels):
        for j in top_influential_segments:
            if j<i*30+30 and j>=i*30:
                results[i][j-i*30]=1
         
    res= pd.DataFrame(results)
    res['electrodo']=np.concatenate([electrodo,electrodo])

    res['hg/b']= ["High Gamma"] * len(electrodes) + ["Beta"] * len(electrodes)
    res['true class']=y_test[id_ecg].argmax()
    res['predicted_class']= predicted_class

    return res

In [ ]:
results=[]
for j in range(X_test.shape[0]):
    results.append(get_segments(  model, X_test, j,electrodes,number_of_top_features=80,num_perturbations = 300,num_slices = 30,channels=8))
    

pd.concat(results).to_csv('results.csv')